# Contrôle qualité au pays de Dracula

Les courbes caractéristiques de composition de Mendenhall sont censées représenter la signature stylistique d’un auteur grâce à une méthode quantitative de telle manière qu’il serait possible d’identifier formellement un texte anonyme. L’idée repose sur un présupposé assez difficile à admettre : le style d’un auteur ne doit varier ni dans le temps ni dans l’espace (les différentes parties d’une même œuvre sont censées adopter la même courbe).

L’algorithme mis en jeu par Mendenhall, rudimentaire pour notre époque, mobilisait des ressources colossales en 1887, lorsqu’il fallait comptabiliser à la main la fréquence d’apparition des mots dans un texte en fonction du nombre de leurs caractères.

Rien de tout cela ici. Au cours de cet exercice, vous mettrez en application des mesures et des tests statistiques afin d’établir ou de rejeter une identité entre deux ensembles de fréquences.

## Présentation de la tâche

Le code ci-dessous importe deux listes de distribution de fréquences des mots dans deux textes de Bram Stoker : *Dracula* (1897) et *The Mystery of the Sea* (1902). Chaque texte a été au préalable divisé en tranches de 5000 mots.

In [ ]:
import pickle

with open('../data/mendenhall.pickle', 'rb') as f:
    novels = pickle.load(f)

dracula = novels['dracula']
mystery = novels['mystery_of_the_sea']

Vérifions la théorie de Mendenhall :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from random import choice

# custom preferences for seaborn
sns.set_theme(rc={
    "figure.figsize": (12, 5),
    "axes.spines.right": False,
    "axes.spines.top": False
})

# a random slice of each text
dracula_slice = choice(range(len(dracula)))
mystery_slice = choice(range(len(mystery)))
dracula_freq = dracula[dracula_slice]
mystery_freq = mystery[mystery_slice]

# get data for each text
dracula_conditions = [ condition for condition, frequencies in dracula_freq ]
dracula_counts = [ frequencies for condition, frequencies in dracula_freq ]
mystery_conditions = [ condition for condition, frequencies in mystery_freq ]
mystery_counts = [ frequencies for condition, frequencies in mystery_freq ]

# plot
figure, ax = plt.subplots()

figure.suptitle("Characteristic curves of composition in two novels by Bram Stoker")

ax.set(
    title="n = 5000 words",
    xlabel="Word length",
    ylabel="Frequency"
)

plt.legend([
    f"Dracula, slice n°{dracula_slice}",
    f"The Mystery of the Sea, slice n°{mystery_slice}"
])

plt.xticks(range(1, 19))
plt.yticks(range(0, 1600, 200))

sns.lineplot(x=dracula_conditions, y=dracula_counts, ax=ax)
sns.lineplot(x=mystery_conditions, y=mystery_counts, ax=ax)

plt.show()

Le phénomène est certes aléatoire – les tranches comparées n’étant ici jamais les mêmes –, vous devriez observer sensiblement les mêmes caractéristiques : un épaulement au niveau des mots de 2 à 4 caractères, puis un éboulis plus ou moins vertigineux avant un aplatissement qui s’amorce à partir des mots de 8 caractères.

Que l’on découvre des similarités dans les courbes de composition de textes d’un même auteur ne choque pas, mais n’en irait-il pas de même si on effectuait la comparaison avec un autre auteur ? Après tout, cela semble naturel que les mots de 1 caractère (*a*, *I*) soient moins nombreux que ceux de 2 (*of*, *my*, *to*, *in*…) étant donné que ces derniers jouent un rôle plus important dans la phrase.

Regardons si la courbe de la 1e tranche du *Frankenstein* de Mary Shelley (1818) montre des ressemblances :

In [ ]:
# data from Frankenstein
frankenstein = novels['frankenstein']
frk_conditions = [ condition for condition, frequencies in frankenstein[0] ]
frk_counts = [ frequencies for condition, frequencies in frankenstein[0] ]

# plot
fig = sns.lineplot(x=frk_conditions, y=frk_counts)

fig.set(
    title="Characteristic curve of composition in Mary Shelley's Frankenstein",
    xlabel="Word length",
    ylabel="Frequency"
)

plt.xticks(range(1, 19))
plt.yticks(range(0, 1600, 200))

plt.show()

Globalement, oui. Dans le détail, on est en droit de se demander si les courbes de composition ne seraient pas plus utiles à départager deux auteurs ou autrices dans une tâche d’attribution.

## Mise en place des données

L’étude que vous allez mener relève du champ de l’exploratoire. Il s’agit d’appliquer des mesures statistiques afin de ressortir une méthode pour comparer des courbes de Mendenhall. Tout commence par le recueillement des données. Pour cela, vous utiliserez la bibliothèque logicielle *Pandas* :

In [ ]:
import pandas as pd

L’opération consiste à consolider les données dans un tableau avec, disposés en lignes, les romans, pour lesquels vous comptabiliserez la somme des fréquences pour chacune des conditions. Créez une variable `dracula_sum_freq` qui recense le nombre total d’occurrences pour chaque condition dans le roman *Dracula*. Vous obtiendrez au final un dictionnaire avec les conditions commes clés et les fréquences comme valeurs :

In [ ]:
# your code here

Comme votre objectif est d’obtenir une courbe caractéristique dans une collection de 5000 mots, établissez une moyenne dans une variable `dracula_mean` :

In [ ]:
# your code here

Importez ce dictionnaire dans un *dataframe* et observez le résultat :

In [ ]:
# dataframe
dracula_df = pd.DataFrame(dracula_mean, index=["Dracula"])

display(dracula_df)

Répétez l’opération pour *The Mystery of the Sea* et *Frankenstein* afin d’obtenir deux variables `mystery_df` et `frankenstein_df` :

In [ ]:
# your code here

Fusionnez l’ensemble dans un nouveau *dataframe* :

In [ ]:
# NA values fixed at 0
df = pd.concat([dracula_df, mystery_df, frankenstein_df]).fillna(0)
# reordering columns
df = df.reindex(sorted(df.columns), axis=1)

display(df)

À ce stade, vous devriez avoir repéré des données aberrantes : il existerait des mots de 54 et de 69 caractères. Supprimez-les en retirant les deux dernières colonnes du *dataframe* :

In [ ]:
df = df.drop(columns=[54, 69])

## Comparaison entre les différentes œuvres

Il est à présent raisonnable de considérer que la moyenne des deux premières lignes du tableau constitue la courbe de composition caractéristique de Bram Stoker quand la dernière ligne vaut pour celle de Mary Shelley :

In [ ]:
stoker = df.iloc[0:2].mean()
shelley = df.iloc[2]

Comparez d’une part les deux écrivain·es et, d’autre part, la courbe idéale de Bram Stoker avec celles caractéristiques de ses deux romans :

In [ ]:
# plot
figure, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

figure.suptitle("Characteristic curves of composition")

# first axis
sns.lineplot(data=stoker, ax=ax1)
sns.lineplot(data=shelley, ax=ax1)
ax1.set(
    title="Comparison between Bram Stoker & Mary Shelley",
    xlabel="Word length",
    ylabel="Frequency"
)
ax1.legend([
    f"Bram Stoker",
    f"Mary Shelley"
])
ax1.set_xticks(range(1, 22))

# second axis
sns.lineplot(data=stoker, ax=ax2)
sns.lineplot(data=df.iloc[0], ax=ax2)
sns.lineplot(data=df.iloc[1], ax=ax2)

ax2.set(
    title="Comparison between two novels by Bram Stoker",
    xlabel="Word length",
    ylabel="Frequency"
)
ax2.legend([
    f"Characteristic curve",
    f"Dracula",
    f"The Mystery of the Sea"
])
ax2.set_xticks(range(1, 22))

plt.show()

Dans cette représentation, les styles de Mary Shelley et de Bram Stoker divergent à deux endroits : au niveau des mots de 2, 3 et 4 caractères, puis entre les mots de 6 et de 13 caractères. L’une des interprétations possibles serait de qualifier le style de Mary Shelley de plus riche en ce qu’il emploie davantage de mots plus longs au détriment des mots très courts. À l’inverse, on pourrait également en conclure que le style de Bram Stoker est plus enlevé, les mots courts servant à dynamiser la phrase.

Pour ce qui est de la comparaison entre les romans de Bram Stoker, rien d’étonnant à ce que les courbes épousent celle qui représente la moyenne entre elles-mêmes. Ce qui serait plus intéressant, ce serait de mesurer leur homogénéité.

## Mise en évidence de l’homogénéité dans le style de Bram Stoker

### Le test de conformité

Grâce à vos calculs, vous avez établi la distribution moyenne des fréquences dans l’œuvre de Bram Stoker, pour chaque tranche de 5000 mots, en vous fondant sur deux de ses romans. Si un nouveau texte devait apparaître, vous seriez en mesure de déterminer s’il respecte cette signature stylistique. Une espèce de contrôle de qualité, en somme. Bien entendu, vous tolérerez une marge d’erreur liée à la créativité et aux innovations de la langue, l’écriture n’étant pas une activité soumise à un processus industriel.

C’est ici qu’intervient le test de conformité. Soit l’hypothèse nulle $H_0$ qui suppose le nouveau texte conforme au style de Bram Stoker, statuant ainsi que les différences observées ne sont pas significatives et qu’elles ne sont au final dues qu’aux fluctuations de l’échantillonnage. L’hypothèse alternative $H_1$ au contraire établira que les différences sont statistiquement significatives.

Dans la pratique, le test revient à appliquer la formule ci-dessous où $\bar{x}$ est la moyenne observée sur le nouveau texte, $\mu$ l’espérance de la signature stylistique, $\sigma$ l’écart-type et $n$ le nombre total de tranches de 5000 mots examinées :

$$
u = \frac{\bar{x} - \mu}{\frac{\sigma}{\sqrt{n}}}
$$

À la fin du calcul, il vous faudra comparer la valeur de *u* avec le risque $\alpha$ retenu, sachant que $\alpha \in ]0, 1[$. Si $\lvert u \rvert > u_\alpha$, alors vous rejetterez $H_0$.

À titre d’exercice, vous appliquerez la formule au *Frankenstein* de Mary Shelley, même si nous savons déjà qu’il n’est pas de la main de Bram Stoker. Avant de commencer, calculez l’écart-type de la distribution des fréquences dans *Frankenstein*.

Vous devrez d’abord calculer la variance puis l’écart-type dans des variables `frk_var` et `frk_std` en appliquant les formules suivantes :

$$
\sigma = \sqrt{V} = \sqrt{\frac{1}{n}\sum^n_{i=1}(x_i - \bar{x})^2}
$$

In [ ]:
# your code here

Si ce n’est pas déjà fait, transformez votre objet en type `pandas.core.series.Series` grâce à la classe `pd.Series()`, puis pensez à la trier par la clé d’indice avec la méthode `.sort_index` :

In [ ]:
# your code here

Il ne vous reste plus qu’à appliquer la formule :

In [ ]:
# your code here

Déterminez à présent si la valeur absolue de *u* est supérieure au seuil $\alpha$ que vous avez fixé (par exemple 0,05) et prononcez-vous sur le rejet de $H_0$ :

In [ ]:
# your code here

### Un indicateur de l’homogénéité

Jetons un œil curieux à la distribution de l’écart-type dans le *Frankenstein* de Mary Shelley :

In [ ]:
fig = sns.barplot(x=frk_std.index, y=frk_std)

fig.set(
    title="Standard deviation of word frequencies by length in Frankenstein",
    xlabel="Word length",
    ylabel="Frequencies"
)

plt.show()

La mesure de l’écart-type permet de représenter la quantité d’erreur par rapport à ce qui aurait été attendu en situation idéale. Le diagramme ci-dessus vous permet par exemple de dire que, dans une tranche de 5000 mots de *Frankenstein*, on a observé que la quantité de mots de deux lettres variait en moyenne de +60 à -60 unités par rapport à la moyenne calculée sur l’ensemble.

Mais est-ce beaucoup ? Cela vous autorise-t-il à dire que l’incertitude est plus forte que pour les mots de 14 caractères ? Pour s’en assurer, il serait préférable de comparer avec les moyennes : un écart-type de 60 mots sur une moyenne de 1000, ce n’est pas la même chose que sur une moyenne de 100 mots.

Le rapport que nous venons d’introduire s’appelle le coefficient de variation. Il s’obtient grâce à la formule :

$$
\text{CV} = \frac{\sigma}{\mu}
$$

Calculez-le pour *Frankenstein* dans une variable `frk_cv` :

In [ ]:
# your code here

L’indicateur obtenu, généralement inférieur à 1, permet de se rendre compte de l’homogénéité dans la composition du roman. Plus il est faible, plus les valeurs sont proches de leur moyenne. On considère qu’en dessous de 0,3 la distribution des mots est homogène dans les tranches de *Frankenstein* pour la longueur considérée.

Un graphique sera plus parlant (la ligne bleue matérialise le seuil d’homogénéité) :

In [ ]:
# plot
figure, ax = plt.subplots()

ax.set(
    title="Homogeneity in composition of Frankenstein",
    xlabel="Word length",
    ylabel="Coefficient of variation"
)

sns.barplot(x=frk_cv.index[:-3], y=frk_cv[:-3], ax=ax)
sns.lineplot(x=range(0, 18), y=[0.3]*18, ax=ax)

plt.show()